### Library used and deeclare data needed

In [20]:
import numpy as np
import csv
import pandas as pd
from numpy.typing import NDArray
from scipy.stats import zscore
from pytictoc import TicToc
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, PredefinedSplit
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
from sklearn.metrics import accuracy_score

In [21]:
kernel_fcn = 'rbf'
opts_csv_fold = 5
with open('./data/algolabels.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            algo_labels = row

y = pd.read_csv('./data/ybin.csv', header=None).values
z = pd.read_csv('./data/z_M.csv', header=None, dtype=np.float64)

ninst, nalgos = y.shape
w = np.ones((ninst, nalgos))

""" Problem 1 - Normalization generate different result with MATLAB """
z_norm = (z-np.mean(z, axis=0))/np.std(z, ddof=1, axis=0)
pd.DataFrame(z_norm).to_csv('z_f.csv', header=None, index=None)
# scaler = StandardScaler().fit(z)

cvcmat = np.zeros((nalgos, 4))


### Prepare k-fold data set

In [31]:
# Read indices
test_fold = np.full(212, -1)

for i in range (opts_csv_fold):
    csv_file = f'../tests/pythia/k-fold/A_2_test_indices_fold_{i + 1}.csv'
    test_indices = pd.read_csv(csv_file, header=None).squeeze().values
    print(test_indices)
    
    adjusted_test_indices = test_indices - 1
    test_fold[adjusted_test_indices] = i

# print(test_fold)

ps = PredefinedSplit(test_fold)

for i, (train_index, test_index) in enumerate(ps.split()):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")


[  1   2   4   5  13  20  27  30  34  35  42  46  49  56  62  69  82  84
  86  87  94  97 111 113 123 126 130 136 142 150 157 160 164 172 179 182
 185 186 190 194 204 206]
[  3   9  14  16  17  18  23  28  37  40  57  63  70  76  99 101 108 110
 112 121 124 125 132 133 144 149 152 161 163 167 169 171 177 181 183 187
 195 196 203 207 208 209 212]
[ 10  21  24  26  41  45  53  54  55  58  61  64  65  67  73  74  77  83
  85  92  93  96 102 103 107 122 127 128 129 138 140 141 154 159 173 174
 175 180 188 189 193 197 205]
[ 12  19  25  29  31  32  33  36  38  39  43  59  68  75  78  88  95  98
 105 115 117 120 134 137 139 143 146 148 151 156 158 162 165 170 184 198
 199 200 201 202 210 211]
[  6   7   8  11  15  22  44  47  48  50  51  52  60  66  71  72  79  80
  81  89  90  91 100 104 106 109 114 116 118 119 131 135 145 147 153 155
 166 168 176 178 191 192]
Fold 0:
  Train: index=[  2   5   6   7   8   9  10  11  13  14  15  16  17  18  20  21  22  23
  24  25  27  28  30  31  32  35  36

In [32]:
# Grid Search on this set for the first algorithm
y_b = y[:, 1]
w_b = w[:, 1]

z_norm = np.ascontiguousarray(z)
y_b = np.ascontiguousarray(y_b)
w_b = np.ascontiguousarray(w_b)

param_grid = {
    'C': np.logspace(-10, 4, base=2, num=30),
    'gamma': np.logspace(-10, 4, base=2, num=30)
}
# By default, the class_weight=None represent equal weight
svm_model = SVC(kernel='rbf', class_weight=None, random_state=0)

        # steps = list()
        # steps.append(('scaler', StandardScaler()))
        # steps.append(('model', svm_model))
        # pipeline = Pipeline(steps=steps)

        # Used for exhaustive search over specified parameter values for the SVM. The param_grid defines 
        # the range over which C and gamma will be tuned.
        # GridSearchCV for optimizing the hyperparameters

        # TODO different c and gamma, same k-fold
        # Unaltimate 
grid_search = GridSearchCV(
            svm_model, param_grid, 
            scoring='accuracy',
            cv=ps, 
            verbose=0
            #, n_jobs=nworkers if nworkers != 0 else None,
        )
grid_search.fit(z_norm, y_b, sample_weight=w_b)
best_svm = grid_search.best_estimator_
pd.DataFrame(grid_search.cv_results_).to_csv("cv_result_.csv")

best_C = grid_search.best_params_['C']
best_g = grid_search.best_params_['gamma']

y_sub = best_svm.predict(z_norm)

aux = confusion_matrix(y_b, y_sub)
print("Best C", best_C)
print("Best gamma", best_g)
print(aux)


Best C 3.0025887646926703
Best gamma 0.14776528839878358
[[ 59  25]
 [  8 120]]


### Check Consistency

In [ ]:
z_norm_M = pd.read_csv('./data/z_norm_M.csv', header=None, dtype=np.float64)
z_norm_P = pd.read_csv('z_f.csv', header=None, dtype=np.float64)

tolerance = 1e-10
are_close = np.isclose(z_norm_M.values, z_norm_P.values, atol=tolerance)
results = are_close.all()

print("Are all elements close within the tolerance level: ", results)
if not results:
    mismatches = np.where(~are_close)
    print("Mismatch found at positions:", mismatches)

Are all elements close within the tolerance level:  True


: 

### Training

In [ ]:
def fit_libsvm(z, y, kkv, kernel_given):
    accuracy= dict()
    for k, v in kkv.items():
        train_index, test_index = v[0], v[1]
        # prepare training data
        x_train = [z[i] for i in train_index]
        y_train = [y[i] for i in train_index]
        # prepare test data
        x_test = [z[i] for i in test_index]
        y_test = [y[i] for i in test_index]
        svm = SVC(kernel=kernel_given, C=1.0, random_state = 0)
        svm.fit(x_train, y_train)
        y_pred = svm.predict(x_test)
        # calculate accuracy
        accuracy[k] = accuracy_score(y_test, y_pred)
        
    return accuracy

: 

In [30]:
def fit_matsvm(z, y, w, skf, kernel_given, params):
    # TODO Set up parallel workers in pool
    

    # Scikit-learn lib need to ensuring data contiguity
    z = np.ascontiguousarray(z)
    y = np.ascontiguousarray(y)
    w = np.ascontiguousarray(w)
    
    # Check if hyperparameter is given by user
    if(np.isnan(params)):

        # Initialize a random number generator
        np.random.seed(0)

        # Retrieve default hyperparameters for fitcsvm and sets the range for the box constraint (C) and kernel scale
        # Define the range for C and gamma in a logarithmic scale
        param_grid = {
        'C': np.logspace(-10, 4, base=2, num=30),
        'gamma': np.logspace(-10, 4, base=2, num=30)
        }

        # By default, the class_weight=None represent equal weight
        svm_model = SVC(kernel=kernel_given, class_weight=None, random_state=0)

        # steps = list()
        # steps.append(('scaler', StandardScaler()))
        # steps.append(('model', svm_model))
        # pipeline = Pipeline(steps=steps)

        # Used for exhaustive search over specified parameter values for the SVM. The param_grid defines 
        # the range over which C and gamma will be tuned.
        # GridSearchCV for optimizing the hyperparameters

        # TODO different c and gamma, same k-fold
        # Unaltimate 
        grid_search = GridSearchCV(
            svm_model, param_grid, 
            scoring='accuracy', # 'roc_auc' 'accuracy'
            cv=skf, 
            verbose=0
            #, n_jobs=nworkers if nworkers != 0 else None,
            )
        grid_search.fit(z, y, sample_weight=w)
        best_svm = grid_search.best_estimator_
        pd.DataFrame(grid_search.cv_results_).to_csv("cv_result_.csv")

        best_C = grid_search.best_params_['C']
        best_g = grid_search.best_params_['gamma']
        
        # best_svm = SVC(kernel=kernel_given,C=best_C, gamma=best_g, class_weight=None, random_state=0)

        # accuracy = np.zeros(skf.n_splits)
        # for i, (train_index, test_index) in enumerate(skf.split(z,y)):
        #     z_train, z_test = z[train_index], z[test_index]
        #     y_train, y_test = y[train_index], y[test_index]

        #     best_svm.fit(z_train,y_train)

        #     # Show the prediction accuracy
        #     y_pred = best_svm.predict(z_test)
        #     accuracy[i] = accuracy_score(y_test, y_pred)
        # print(f"mean accuracy is {np.mean(accuracy)}")
    
        # OPT1: 
        # Fit a probability calibration model with trained SVM
        # print(z.shape, y.shape, w.shape)
        calibrator = CalibratedClassifierCV(best_svm, cv='prefit', method='sigmoid')
        calibrator.fit(z, y, sample_weight=w)

        # OPT2: retrain
        # calibrator = CalibratedClassifierCV(best_svm, cv=skf, method='sigmoid')
        # calibrator.fit(z_norm, y, sample_weight=w)

        best_C = grid_search.best_params_['C']
        best_g = grid_search.best_params_['gamma']

        y_sub = best_svm.predict(z)
        p_sub = calibrator.predict_proba(z)

        # Making predictions on the same data to simulate resubstitution prediction
        y_hat = y_sub
        p_hat = p_sub
        
        print("Best C:", best_C)
        print("Best gamma:", best_g)


    return best_svm, y_sub, p_sub, y_hat, p_hat, best_C, best_g

### MATSVM


In [28]:
t = TicToc()
t.tic()
# skf = StratifiedKFold(n_splits = opts_csv_fold, shuffle=True, random_state=0)

for i in range(nalgos):
    t_inner = TicToc()
    t_inner.tic()

    state = np.random.get_state()
    np.random.seed(0)  # equivalent to MATLAB's rng('default') ?

    # y_b = [row[i] for row in y]
    y_b = y[:, i]

    # Problem: Not exactly the same for each run ?????
    # https://stackoverflow.com/questions/73527928/does-stratifiedkfold-splits-the-same-each-time-a-for-loop-is-called
    skf = StratifiedKFold(n_splits = opts_csv_fold, shuffle=True, random_state=0)
    # skf = KFold(n_splits = opts_csv_fold, shuffle = True, random_state = 0)

    # fit_matsvm(z_norm, y_b, w[:, i], skf, kernel_fcn, np.nan)

    best_svm, y_sub, p_sub, y_hat, p_hat, best_C, best_g = fit_matsvm(z_norm, y_b, w[:, i], skf, kernel_fcn, np.nan)
    aux = confusion_matrix(y_b, y_sub)
    print(aux)
    # np.prod(aux.shape) != 4 is False
    cvcmat[i, :] = aux.flatten()

    models_left = nalgos - (i + 1)
    print(f"    -> PYTHIA has trained a model for {algo_labels[i]}, there are {models_left} models left to train.")

    print(f"      -> Elapsed time: {t_inner.tocvalue():.2f}s")

    

Best C: 1.1003260089454092
Best gamma: 11.449727669768906
[[ 73  10]
 [  2 127]]
    -> PYTHIA has trained a model for NB, there are 9 models left to train.
      -> Elapsed time: 19.08s
Best C: 3.0025887646926703
Best gamma: 0.014200336875591593
[[ 59  25]
 [ 10 118]]
    -> PYTHIA has trained a model for LDA, there are 8 models left to train.
      -> Elapsed time: 18.88s
Best C: 1.1003260089454092
Best gamma: 0.05414993620051815
[[118  19]
 [ 21  54]]
    -> PYTHIA has trained a model for QDA, there are 7 models left to train.
      -> Elapsed time: 18.38s
Best C: 11.449727669768906
Best gamma: 0.010161874377781294
[[ 54   7]
 [  9 142]]
    -> PYTHIA has trained a model for CART, there are 6 models left to train.
      -> Elapsed time: 16.66s
Best C: 4.195857021292137
Best gamma: 0.0756698328726082
[[ 50   8]
 [ 10 144]]
    -> PYTHIA has trained a model for J48, there are 5 models left to train.
      -> Elapsed time: 16.53s
Best C: 1.537610033259584
Best gamma: 0.1057420194506833

### Estimation

In [29]:
  
tn, fp, fn, tp = cvcmat[:, 0], cvcmat[:, 1], cvcmat[:, 2], cvcmat[:, 3]
print(tn.dtype)  
print(tn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tp + tn) / ninst

print(precision)
print(recall)
print(accuracy)

float64
[ 73.  59. 118.  54.  50.  51.  51.  86.  57.  48.]
[0.9270073  0.82517483 0.73972603 0.95302013 0.94736842 0.95394737
 0.93506494 0.875      0.96621622 0.87820513]
[0.98449612 0.921875   0.72       0.94039735 0.93506494 0.94155844
 0.95364238 0.875      0.95333333 0.94482759]
[0.94339623 0.83490566 0.81132075 0.9245283  0.91509434 0.9245283
 0.91981132 0.86792453 0.94339623 0.87264151]


### LIBSVM

In [ ]:
y = pd.read_csv('./data/ybin.csv')
y = y.values.tolist()

z = pd.read_csv('./data/z.csv').values.tolist()
z_norm = zscore(z, axis = 0, ddof = 1)

for i in range(nalgos):
    t_inner = TicToc()
    t_inner.tic()

    state = np.random.get_state()
    np.random.seed(0)  # equivalent to MATLAB's rng('default') ?

    # REQUIRE: Test case for validation the result
    y_b = [row[i] for row in y]
    # y_b = y[:, i]
    skf = StratifiedKFold(n_splits = opts_csv_fold, shuffle = True, random_state = 0)
    
    kkv= dict()
    for i, (train_index, test_index) in enumerate(skf.split(np.zeros(len(y_b)), y_b)):
        kkv[i] = [train_index.tolist(), test_index.tolist()]
    
    # start training using svm
    svm_res = fit_libsvm(z_norm, y_b, kkv, kernel_fcn)

    # visualise accuracy score
for k, v in svm_res.items():
    print(f'{k} fold: accuracy score = {v}')

0 fold: accuracy score = 0.8837209302325582
1 fold: accuracy score = 0.9047619047619048
2 fold: accuracy score = 0.8333333333333334
3 fold: accuracy score = 0.8333333333333334
4 fold: accuracy score = 0.8571428571428571


: 

In [ ]:
## FITLIBSVM